In [1]:
from models.fcn.fcn import FCN
from models.classifiers.linear import LinearClassifier

import torch.nn as nn
import torch

from utils.data import load_data, transfer_labels, normalize_per_series
from utils.data import UCRDataset, fill_nan_value
from utils.kfold import k_fold
from utils.save_classification_results import save_cls_result

from torch.utils.data import DataLoader
import tqdm

import os
import time

from utils.evaluate import evaluate

import numpy as np
import pandas as pd


# Build Dataset

In [2]:
train_test_dataset, train_test_target, num_classes = load_data(data_root='data/UCR_TS_Archive_2015',  dataset='Adiac')
train_test_target = transfer_labels(train_test_target)

num_classes = num_classes
seq_len = train_test_dataset.shape[1]
 

In [3]:
train_test_dataset

array([[1.598 , 1.5994, 1.5705, ..., 1.5642, 1.5709, 1.5929],
       [1.7011, 1.6706, 1.6189, ..., 1.5197, 1.6025, 1.6702],
       [1.7223, 1.6953, 1.6569, ..., 1.6418, 1.695 , 1.7085],
       ...,
       [1.652 , 1.6968, 1.7006, ..., 1.4993, 1.5557, 1.6204],
       [1.3987, 1.2934, 1.1888, ..., 1.6363, 1.5626, 1.4605],
       [1.7272, 1.7284, 1.6938, ..., 1.6273, 1.6753, 1.6989]],
      dtype=float32)

# Model

In [4]:
model_fcn = FCN(input_size=1, num_classes=2)
linear_classifier = LinearClassifier(128, 2)

loss = nn.CrossEntropyLoss()

model_init_state = model_fcn.state_dict()
classifier_init_state = linear_classifier.state_dict()

optimizer = torch.optim.Adam([{'params': model_fcn.parameters()}, {'params': linear_classifier.parameters()}],
                                     lr=0.001, weight_decay=0.0)

train_test_dataset = normalize_per_series(train_test_dataset)


In [5]:
train_test_dataset

array([[1.6025561, 1.6039602, 1.5749776, ..., 1.5686598, 1.5753788,
        1.5974416],
       [1.7059497, 1.6753628, 1.6235152, ..., 1.5240326, 1.6070685,
        1.6749616],
       [1.7272137, 1.7001365, 1.661627 , ..., 1.646484 , 1.6998358,
        1.7133743],
       ...,
       [1.6567166, 1.7016445, 1.7054554, ..., 1.5035807, 1.5601417,
        1.6250263],
       [1.4026968, 1.2970963, 1.1921976, ..., 1.6409751, 1.5670648,
        1.4646733],
       [1.732125 , 1.7333285, 1.6986297, ..., 1.6319402, 1.6800771,
        1.7037443]], dtype=float32)

# Loading train and test dataset

In [6]:
device = 'mps'
batch_size = 8
train_set = UCRDataset(torch.from_numpy(train_test_dataset).to(device),
                               torch.from_numpy(train_test_target).to(device).to(torch.int64))
train_loader = DataLoader(train_set, batch_size=batch_size, num_workers=0)

# Adiac dataset is univariate. meaning it has only one channel and 176 timesteps
print(f"Train loader shape: {train_loader.dataset.dataset.shape}")


Train loader shape: torch.Size([781, 1, 176])


# Pre-Training the model

In [7]:

last_loss = float('inf')
stop_count = 0
increase_count = 0

min_loss = float('inf')
min_epoch = 0
model_to_save = None

save_dir = "models_archive/pre_trained"
pre_training_dataset = "Adiac"  # Dataset Name

model_fcn.to(device)
linear_classifier.to(device)

num_steps = train_set.__len__() // batch_size
for epoch in range(1, 101):
    
    if stop_count == 50 or increase_count == 50:
        print("model convergent at epoch {}, early stopping.".format(epoch))
        break
    
    epoch_loss = 0
    epoch_accuracy = 0
    
    model_fcn.train()
    linear_classifier.train()
    for batch_idx, (data, target) in tqdm.tqdm(enumerate(train_loader), total=len(train_loader), desc="Pre-training FCN"):
        optimizer.zero_grad()
        predictions = model_fcn(data)
        predictions = linear_classifier(predictions)
        
        step_loss = loss(predictions, target)
        step_loss.backward()
        optimizer.step()
        
        epoch_loss += step_loss.item()
        epoch_accuracy += torch.sum(torch.argmax(predictions.data, axis=1) == target) / len(target)
        
    epoch_loss /= num_steps
    if abs(epoch_loss - last_loss) <= 1e-4:
        stop_count += 1
    else:
        stop_count = 0

    if epoch_loss > last_loss:
        increase_count += 1
    else:
        increase_count = 0
        
    last_loss = epoch_loss
    if epoch_loss < min_loss:
        min_loss = epoch_loss
        min_epoch = epoch
        model_to_save = model_fcn.state_dict()
        classifier_to_save = linear_classifier.state_dict()

    epoch_accuracy /= num_steps
    if epoch % 100 == 0:
        print("epoch : {}, loss : {}, accuracy : {}".format(epoch, epoch_loss, epoch_accuracy))
        torch.save(model_to_save, os.path.join(save_dir, pre_training_dataset, 'pretrain_weights.pt'))
        torch.save(classifier_to_save, os.path.join(save_dir, pre_training_dataset, 'classifier_weights.pt'))

print('{} finished pretrain, with min loss {} at epoch {}'.format(pre_training_dataset, min_loss, min_epoch))
torch.save(model_to_save, os.path.join(save_dir, pre_training_dataset, 'pretrain_weights.pt'))


Pre-training FCN:   0%|          | 0/98 [00:00<?, ?it/s]/Users/neereshkumarperla/miniconda3/lib/python3.10/site-packages/torch/nn/modules/conv.py:306: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at /private/var/folders/k1/30mswbxs7r1g6zwn8y4fyt500000gp/T/abs_5ae0635zuj/croot/pytorch-select_1700511177724/work/aten/src/ATen/native/Convolution.cpp:1009.)
  return F.conv1d(input, weight, bias, self.stride,
Pre-training FCN: 100%|██████████| 98/98 [00:00<00:00, 209.64it/s]

model convergent at epoch 60, early stopping.
Adiac finished pretrain, with min loss 0.017359792955757417 at epoch 59


# Finetuning on another dataset

In [8]:
# Loading weights
torch.load(f"models_archive/pre_trained/{pre_training_dataset}/classifier_weights.pt")


OrderedDict([('dense.weight',
              tensor([[-1.0743e-01, -6.0665e-02,  5.1999e-05,  1.1033e-01, -6.8118e-02,
                        1.3369e-01, -1.8848e-02,  1.3225e-01,  9.0464e-02,  1.3808e-01,
                       -6.3941e-02,  9.6029e-02,  2.9341e-02, -9.8433e-02, -5.6932e-02,
                       -4.4407e-02,  1.1042e-01, -1.3372e-01,  1.3426e-01, -1.2734e-01,
                        1.0477e-01,  8.7777e-02,  1.2434e-01, -4.0225e-02, -8.2705e-02,
                       -6.6935e-02,  1.3251e-01,  1.3236e-02,  1.6723e-02, -7.8169e-02,
                        4.4514e-02, -3.6628e-02,  3.6761e-02,  8.1834e-02,  1.2099e-01,
                        1.3229e-01, -1.1962e-01,  8.5638e-02,  1.1296e-01, -8.4369e-02,
                        1.9186e-02, -6.2912e-02,  2.8851e-02,  8.5476e-02,  1.0799e-02,
                       -1.1907e-01,  9.0666e-02,  4.3757e-02, -9.7290e-02,  1.3246e-01,
                       -1.1541e-01, -6.1207e-02,  1.1963e-01, -7.2790e-02, -8.8357e-02,
  

## Loading Target Dataset

In [9]:
finetuned_dataset = 'ArrowHead'
train_test_dataset, train_test_target, num_classes = load_data(data_root='data/UCR_TS_Archive_2015', 
                                                               dataset=finetuned_dataset)
train_test_target = transfer_labels(train_test_target)
 

In [10]:
train_datasets, train_targets, val_datasets, val_targets, test_datasets, test_targets = k_fold(train_test_dataset, train_test_target)


In [11]:
losses = []
test_accuracies = []
train_time = 0.0
end_val_epochs = []

for i, train_dataset in enumerate(train_datasets):
    t = time.time()
    model_fcn.load_state_dict(torch.load(os.path.join(save_dir, 'Adiac', 'pretrain_weights.pt')))
    linear_classifier.load_state_dict(classifier_init_state)
    print('{} fold start training and evaluate'.format(i))
    max_accuracy = 0
    
    train_target = train_targets[i]
    val_dataset = val_datasets[i]
    val_target = val_targets[i]

    test_dataset = test_datasets[i]
    test_target = test_targets[i]
    
    train_dataset, val_dataset, test_dataset = fill_nan_value(train_dataset, val_dataset, test_dataset)
    train_dataset = normalize_per_series(train_dataset)
    val_dataset = normalize_per_series(val_dataset)
    test_dataset = normalize_per_series(test_dataset)
    
    train_set = UCRDataset(torch.from_numpy(train_dataset).to(device), 
                           torch.from_numpy(train_target).to(device).to(torch.int64))
    val_set = UCRDataset(torch.from_numpy(val_dataset).to(device),
                         torch.from_numpy(val_target).to(device).to(torch.int64))
    test_set = UCRDataset(torch.from_numpy(test_dataset).to(device),
                          torch.from_numpy(test_target).to(device).to(torch.int64))

    train_loader = DataLoader(train_set, batch_size=batch_size, num_workers=0, drop_last=True)
    val_loader = DataLoader(val_set, batch_size=batch_size, num_workers=0)
    test_loader = DataLoader(test_set, batch_size=batch_size, num_workers=0)
    
    train_loss = []
    train_accuracy = []
    num_steps = epoch // batch_size

    last_loss = float('inf')
    stop_count = 0
    increase_count = 0

    test_accuracy = 0
    min_val_loss = float('inf')
    end_val_epoch = 0
    
    num_steps = train_set.__len__() // batch_size
    for epoch in range(1, 101):
        if stop_count == 50 or increase_count == 50:
            print('model convergent at epoch {}, early stopping'.format(epoch))
            break

        epoch_train_loss = 0
        epoch_train_acc = 0
        model_fcn.train()
        linear_classifier.train()
        for x, y in train_loader:
            optimizer.zero_grad()
            pred = model_fcn(x)
            pred = linear_classifier(pred)

            step_loss = loss(pred, y)
            step_loss.backward()
            optimizer.step()

            epoch_train_loss += step_loss.item()
            epoch_train_acc += torch.sum(torch.argmax(pred.data, axis=1) == y) / len(y)

        epoch_train_loss /= num_steps
        epoch_train_acc /= num_steps

        model_fcn.eval()
        linear_classifier.eval()
        val_loss, val_accu = evaluate(val_loader, model_fcn, linear_classifier, loss, device)
        if min_val_loss > val_loss:
            min_val_loss = val_loss
            end_val_epoch = epoch
            test_loss, test_accuracy = evaluate(test_loader, model_fcn, linear_classifier, loss, device)

        if epoch % 100 == 0:
            print("epoch : {}, train loss: {} , train accuracy : {}, \nval loss : {}, val accuracy : {}, \ntest loss : {}, test accuracy : {}".format(epoch, epoch_train_loss, epoch_train_acc, val_loss, val_accu, test_loss, test_accuracy))

        if abs(last_loss - val_loss) <= 1e-4:
            stop_count += 1
        else:
            stop_count = 0

        if val_loss > last_loss:
            increase_count += 1
        else:
            increase_count = 0

        last_loss = val_loss
    test_accuracies.append(test_accuracy)
    end_val_epochs.append(end_val_epoch)
    t = time.time() - t
    train_time += t

    print('{} fold finish training'.format(i))

test_accuracies = torch.Tensor(test_accuracies)
end_val_epochs = np.array(end_val_epochs)

save_cls_result(save_dir='results', save_csv_name='FCN', dataset_name=f'{pre_training_dataset}_{finetuned_dataset}',
                test_accu=torch.mean(test_accuracies), test_std=torch.std(test_accuracies),
                train_time=train_time / 5, end_val_epoch=np.mean(end_val_epochs))
print('Done!')


0 fold start training and evaluate
epoch : 100, train loss: 0.38766416609287263 , train accuracy : 0.6083333492279053, 
val loss : 0.04485278824965159, val accuracy : 0.5476190447807312, 
test loss : 0.03937501962794814, test accuracy : 0.6279069781303406
0 fold finish training
1 fold start training and evaluate
epoch : 100, train loss: 0.38616437117258706 , train accuracy : 0.6000000238418579, 
val loss : 0.0371894363400548, val accuracy : 0.6511628031730652, 
test loss : 0.039125887410981317, test accuracy : 0.6428571343421936
1 fold finish training
2 fold start training and evaluate
epoch : 100, train loss: 0.3683502991994222 , train accuracy : 0.6083333492279053, 
val loss : 0.05686238200165505, val accuracy : 0.39534884691238403, 
test loss : 0.04233402120215552, test accuracy : 0.5476190447807312
2 fold finish training
3 fold start training and evaluate
epoch : 100, train loss: 0.4028823475042979 , train accuracy : 0.5583333373069763, 
val loss : 0.04579668405444123, val accuracy

# Results

In [12]:
pd.read_csv('results/FCN_cls_result.csv')

,id,dataset_name,test_accuracy,test_std,train_time,end_val_epoch,seeds
0,0,Adiac_ArrowHead,0.6112,0.0562,8.1199,97.2,42
1,1,Adiac_ArrowHead,0.5782,0.0260,7.8860,93.0,42
2,2,Adiac_ArrowHead,0.5827,0.0512,8.1239,85.8,42
